# CNN Visualization
1. Activation maps
2. Filter using gradient ascent
3. Deep dream using gradient ascent

In [1]:
%env CUDA_VISIBLE_DEVICES = ""

env: CUDA_VISIBLE_DEVICES=""


In [2]:
import os 
import tensorflow as tf 
import tensorflow.keras as keras
tf.config.gpu.set_per_process_memory_growth(True)
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()

Use a pretrained model like VGG16 and VGG19 trained on ImageNet to do the task.

In [3]:
vgg19 = keras.applications.VGG19(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

In [4]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
version = 1
export_path = './model/{}'.format(version)
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)

# Build the signature_def_map.
inputs = tf.compat.v1.saved_model.build_tensor_info(vgg19.input)
add_variable = False

l_ts = []
for l in vgg19.layers[1:]:
    base64_img = tf.strings.as_string(l.output)
    base64_img = tf.reshape(base64_img, [-1])
    base64_img = tf.strings.reduce_join(base64_img, separator=',')
    base64_img = tf.io.encode_base64(base64_img)
    shape = tf.strings.as_string(l.output.shape[1:])
    shape = tf.strings.reduce_join(shape, separator=',')
    msg = tf.strings.join([shape, base64_img], separator='|')
    l_ts.append((l.name, tf.compat.v1.saved_model.build_tensor_info(msg)))
l_ts = dict(l_ts)
signature = (
    # method name must be CLASSIFY_METHOD_NAME, PREDICT_METHOD_NAME or REGRESS_METHOD_NAME
    tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
        inputs={'input': inputs},
        outputs=l_ts,
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME 
    )
)


builder.add_meta_graph_and_variables(
    tf.compat.v1.keras.backend.get_session(), [tf.compat.v1.saved_model.tag_constants.SERVING],
    signature_def_map={'activation':signature,},
    strip_default_attrs=True
)
add_variable = True
builder.save()

W0712 19:58:46.452870 139914048964416 deprecation.py:323] From <ipython-input-5-9b25fa8a44bc>:6: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


b'./model/1/saved_model.pb'

In [6]:
!saved_model_cli show --dir ./model/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['activation']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 224, 224, 3)
        name: input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['block1_conv1'] tensor_info:
        dtype: DT_STRING
        shape: ()
        name: StringJoin:0
    outputs['block1_conv2'] tensor_info:
        dtype: DT_STRING
        shape: ()
        name: StringJoin_1:0
    outputs['block1_pool'] tensor_info:
        dtype: DT_STRING
        shape: ()
        name: StringJoin_2:0
    outputs['block2_conv1'] tensor_info:
        dtype: DT_STRING
        shape: ()
        name: StringJoin_3:0
    outputs['block2_conv2'] tensor_info:
        dtype: DT_STRING
        shape: ()
        name: StringJoin_4:0
    outputs['block2_pool'] tensor_info:
        dtype: DT_STRING
        shape: (